# APSG tutorial - Part 2

In [1]:
from apsg import *

## Matrix like classes and tensors
**APSG** provides matrix-like classes to work with tensor quantities used commonly in structural geology analysis. It includes `DefGrad3` and `VelGrad3` for deformation and velocity gradient, `Stress3` for stress tensor, `Ellipsoid` for quadratic forms and `Ortensor3` for orientation tensor. All these classes support common matrix mathematical operations and provide basic methods and properties.

All matrix-like objects could be created either by passing nested list or tuple or providing individual components to class method `from_comp`

In [2]:
F = defgrad([[2, 0, 1], [0, 1, 0], [0, 0, 0.5]])
F

DeformationGradient3
[[2.  0.  1. ]
 [0.  1.  0. ]
 [0.  0.  0.5]]

In [3]:
F = defgrad.from_comp(xx=2, zz=0.5, xz=1)
F

DeformationGradient3
[[2.  0.  1. ]
 [0.  1.  0. ]
 [0.  0.  0.5]]

For multiplifications of matrix or vectors we have to use matmul `@` operator

In [4]:
v = vec('z') # unit-length vector in direction af axis z
u = F @ v
u

Vector3(1, 0, 0.5)

`I` property returns inverse matrix

In [5]:
F.I @ u

Vector3(0, 0, 1)

To transpose matrix, we can use `T` property and for multiplification we have to use `@` operator

In [6]:
F.T @ F

DeformationGradient3
[[4.   0.   2.  ]
 [0.   1.   0.  ]
 [2.   0.   1.25]]

In [7]:
v @ F.T @ F @ v

1.25

Eigenvalues and eigenvectors could be obtained by methods `eigenvalues` and `eigenvectors`. Individual eigenvalues and eigen vectors could be accessed by properties `E1`, `E2`, `E3` and `V1`, `V2`, `V3`

#### Deformation gradient and rotations

Deformation gradient `DeformationGradient3` could describe distorsion, dilation and rigid-body rotation. All **APSG** features provides `transform` method which transform then using provided deformation gradient.

The rigid-body rotation could be either extracted from deformation gradient using `R` method:

In [8]:
R = F.R
R

DeformationGradient3
[[ 0.928  0.     0.371]
 [ 0.     1.     0.   ]
 [-0.371  0.     0.928]]

or could be created of one of the class methods like `from_axisangle`, defining axis of rotation and angle

In [9]:
R = defgrad.from_axisangle(lin(120, 50), 60)
R

DeformationGradient3
[[ 0.552 -0.753  0.359]
 [ 0.574  0.655  0.492]
 [-0.605 -0.065  0.793]]

`from_two_vectors`, where axis of rotation is perpendicular to both vectors and angle is angle of vectors

In [10]:
R = defgrad.from_two_vectors(lin(120, 50), lin(270, 80))
R

DeformationGradient3
[[ 0.938  0.074  0.339]
 [ 0.186  0.718 -0.671]
 [-0.294  0.692  0.66 ]]

In [11]:
lin(120, 50).transform(R)

L:270/80

or by `from_vectors_axis`, where `axis` do not need to by perpendicular to vectors. Note that rotation axis needs to be adjusted to provide correct rotation of vector.

In [12]:
R = defgrad.from_vectors_axis(lin(45,30), lin(135, 30), lin(90, 70))
R

DeformationGradient3
[[-0.393 -0.864  0.315]
 [ 0.864 -0.23   0.448]
 [-0.315  0.448  0.837]]

In [13]:
lin(45,30).transform(R)

L:135/30

In [14]:
a, ang = R.axisangle()
print(lin(a), ang)

L:90/70 113.11571469196132


`from_two_pairs` method, to describe rotation between two coordinate systems. Note that pair define X axis as lineation vector and Z axis as foliation vector.

In [15]:
p1 = pair(150, 60, 90, 40)
p2 = pair(45, 30, 10, 25)
R = defgrad.from_two_pairs(p1, p2)
R

DeformationGradient3
[[-0.071  0.97   0.234]
 [-0.874 -0.174  0.453]
 [ 0.48  -0.173  0.86 ]]

In [16]:
p1.transform(R)

P:45/30-10/25

### Ellipsoid

In deformation analysis, the quadratic forms are represented by `Ellipsoid` class. It could be used to represents either ellipsoid objects or finite strain ellipsoid.

It provides additional methods and properties including `lambda1`, `lambda2` and `lambda3` for square-root of eigenvalues, Woodcock's `shape` and `strength`, `k`, `K`, `d` and `D` for Flinn's and Ramsay symmetries and intensities, `lode` for Lode's parameter etc. For more check documentation. Eigenvectors could be also represented by linear or planar features using properties `eigenlins` and `eigenfols`.

We can create `Ellipsoid` object similarly to `Matrix3` (note that only components of upper triangular part are available in `from_comp` method due to matrix symmetry), or you can use aditional class methods `from_defgrad` and `from_stretch`.

In [17]:
B = ellipsoid.from_defgrad(F)  # Finger deformation tensor
B

Ellipsoid
[[5.   0.   0.5 ]
 [0.   1.   0.  ]
 [0.5  0.   0.25]]
(S1:2.25, S2:1, S3:0.445)

In above example, the Finger deformation tensor `B` represents finite strain ellipsoid reulting from deformation described by deformation gradient `F`. We can explore several parameters:

In [18]:
print(f'Principal stretches: Sx={B.S1}, Sy={B.S2}, Sz={B.S3}')
print(f'Principal strain ratios: Rxy={B.Rxy}, Ryz={B.Ryz}')
print(f"Flinn's finite strain parameters: d={B.d}, k={B.k}")
print(f"Ramsay's finite strain parameters: d={B.D}, k={B.K}")
print(f"Woodcock's parameters: strength={B.strength}, shape={B.shape}")
print(f"Watterson's strain intesity: s{B.r}")
print(f"Nadai's natural octahedral unit shear: {B.goct}")
print(f"Nadai's natural octahedral unit strain: {B.eoct}")
print(f"Lode's parameter: {B.lode}")

Principal stretches: Sx=2.2476790206496235, Sy=1.0, Sz=0.44490338291762865
Principal strain ratios: Rxy=2.2476790206496235, Ryz=2.2476790206496235
Flinn's finite strain parameters: d=1.7644845924910786, k=1.0
Ramsay's finite strain parameters: d=1.3118699860194973, k=1.0
Woodcock's parameters: strength=1.6197962748565002, shape=1.0
Watterson's strain intesity: s3.495358041299247
Nadai's natural octahedral unit shear: 1.3225581202197996
Nadai's natural octahedral unit strain: 1.14536893009174
Lode's parameter: 0.0


In [19]:
C = ellipsoid.from_defgrad(F, 'right')  # Green's deformation tensor
C

Ellipsoid
[[4.   0.   2.  ]
 [0.   1.   0.  ]
 [2.   0.   1.25]]
(S1:2.25, S2:1, S3:0.445)

In [20]:
v @ C @ v

1.25

### Orientation tensor
`OrientationTensor3` class represents orientation tensor of set of vectors, linear or planar features. In adition to `Ellipsoid` methods and properties, it provides properties to describe orientation distribution, e.g. Vollmer's `P`, `G`, `R` and `B` indexes, `Intensity` for Lisle intensity index and `MAD` for approximate angular deviation.

In [21]:
l = linset.random_fisher(position=lin(120,40))
ot = l.ortensor()
# or
ot = ortensor.from_features(l)
ot

OrientationTensor3
[[ 0.179 -0.211 -0.207]
 [-0.211  0.442  0.369]
 [-0.207  0.369  0.379]]
(S1:0.95, S2:0.249, S3:0.19)

In [22]:
ot.eigenvalues()

array([0.90214885, 0.0618688 , 0.03598235])

In [23]:
ot.eigenvectors()

(Vector3(-0.379, 0.679, 0.629),
 Vector3(0.859, 0.511, -0.034),
 Vector3(0.344, -0.527, 0.777))

In [24]:
ot.kind

'LLS'

The instances of `Stress3`, `Ellipsoid` and `OrientationTensor3` also provides `eigenlins` and `eigenfols` properties to represent principal axes and planes

In [25]:
ot.eigenlins()

(L:119/39, L:211/2, L:303/51)

In [26]:
ot.eigenfols()

(S:299/51, S:31/88, S:123/39)

In [27]:
ot.strength, ot.shape

(1.6108755340150451, 4.944326005763203)

In [28]:
ot.k, ot.d

(9.055223646958394, 2.8357269066656983)

In [29]:
ot.K, ot.D

(4.944326005763203, 1.8687206859419483)

In [30]:
ot.P, ot.G, ot.R

(0.8402800455115945, 0.05177291320399845, 0.10794704128440638)

In [31]:
ot.MAD

18.228757699048067